# Test the API for historical stock price data

In [8]:
import yfinance as yf
import numpy as np
from datetime import datetime

# A simple example for Apple 

In [2]:
aapl = yf.Ticker('AAPL')
print(aapl)

yfinance.Ticker object <AAPL>


In [3]:
# get stock info
aapl.info

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 147000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand r

In [4]:
# get quote/price history
df_aapl_quote_history = aapl.history(interval='1mo', start='2010-01-01', end='2021-09-23')
print(df_aapl_quote_history)

                  Open        High         Low       Close        Volume  \
Date                                                                       
2010-01-01    6.544802    6.611038    5.833990    5.889494  1.516899e+10   
2010-02-01    5.899002    6.291512    5.852391    6.274646  1.077608e+10   
2010-03-01    6.309297    7.282294    6.300098    7.206244  1.215417e+10   
2010-04-01    7.280148    8.354951    7.137250    8.006292  1.236713e+10   
2010-05-01    8.090619    8.214506    6.109975    7.877192  1.808265e+10   
...                ...         ...         ...         ...           ...   
2021-05-07         NaN         NaN         NaN         NaN           NaN   
2021-06-01  124.892880  137.204435  122.945792  136.755112  1.606590e+09   
2021-07-01  136.395644  149.775590  135.556889  145.641785  1.918976e+09   
2021-08-01  146.141042  153.260380  144.283824  151.602859  1.460744e+09   
2021-09-01  152.830002  157.259995  141.270004  141.500000  1.797466e+09   

           

### <font color=blue>Available paramaters for the history() method are:</font>

- **period**: data period to download (Either Use period parameter or use start and end) Valid periods are: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
- **interval**: data interval (intraday data cannot extend last 60 days) Valid intervals are: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo
- **start**: If not using period - Download start date string (YYYY-MM-DD) or datetime.
- **end**: If not using period - Download end date string (YYYY-MM-DD) or datetime.
- **prepost**: Include Pre and Post market data in results? (Default is False)
- **auto_adjust**: Adjust all OHLC automatically? (Default is True)
- **actions**: Download stock dividends and stock splits events? (Default is True)

In [5]:
# create a "datetime": "close price" dictionary from the dataframe of historical price data
payload_aapl_quote_history = dict(df_aapl_quote_history['Close'])
payload_aapl_quote_history

{Timestamp('2010-01-01 00:00:00'): 5.889493942260742,
 Timestamp('2010-02-01 00:00:00'): 6.274646282196045,
 Timestamp('2010-03-01 00:00:00'): 7.206244468688965,
 Timestamp('2010-04-01 00:00:00'): 8.006292343139648,
 Timestamp('2010-05-01 00:00:00'): 7.87719202041626,
 Timestamp('2010-06-01 00:00:00'): 7.713136196136475,
 Timestamp('2010-07-01 00:00:00'): 7.8885369300842285,
 Timestamp('2010-08-01 00:00:00'): 7.454631328582764,
 Timestamp('2010-09-01 00:00:00'): 8.701157569885254,
 Timestamp('2010-10-01 00:00:00'): 9.229513168334961,
 Timestamp('2010-11-01 00:00:00'): 9.541375160217285,
 Timestamp('2010-12-01 00:00:00'): 9.891261100769043,
 Timestamp('2011-01-01 00:00:00'): 10.405207633972168,
 Timestamp('2011-02-01 00:00:00'): 10.831138610839844,
 Timestamp('2011-03-01 00:00:00'): 10.68701457977295,
 Timestamp('2011-04-01 00:00:00'): 10.73669147491455,
 Timestamp('2011-05-01 00:00:00'): 10.66616153717041,
 Timestamp('2011-06-01 00:00:00'): 10.293275833129883,
 Timestamp('2011-07-01 00

### <font color=blue>There are issues with the dataframe-converted dictionary above:</font>

- **"nan" values**: it seems like those records with a "nan" price value can be simply removed;
- **Pandas Timestamp objects**: I don't like them, so will convert those to python's native datetime objects

I'll use a helper function to handle the json payload creation from the API returned dataframe of historical price data

In [7]:
# first delete the dirty json payload created above
del(payload_aapl_quote_history)

In [20]:
def get_payload_json_quote_history_from_yfinance_df(df_quote_history, header='Close'):
    """
    This helper function gets a cleaned-up payload json/dict from the dataframe of 
    historical quotes, which is returned by yfinance api (yfinance.Ticker('<symbol>').history)
    """
    
    _payload_json = {}
    _df = df_quote_history[header]
    
    # loop through all timestamps in the data:
    for timestamp in dict(_df):
        if not np.isnan(_df[timestamp]):
            _payload_json[timestamp.to_pydatetime()] = _df[timestamp]
            
    return _payload_json

# test
payload_aapl_quote_history = get_payload_json_quote_history_from_yfinance_df(df_aapl_quote_history)
payload_aapl_quote_history

{datetime.datetime(2010, 1, 1, 0, 0): 5.889493942260742,
 datetime.datetime(2010, 2, 1, 0, 0): 6.274646282196045,
 datetime.datetime(2010, 3, 1, 0, 0): 7.206244468688965,
 datetime.datetime(2010, 4, 1, 0, 0): 8.006292343139648,
 datetime.datetime(2010, 5, 1, 0, 0): 7.87719202041626,
 datetime.datetime(2010, 6, 1, 0, 0): 7.713136196136475,
 datetime.datetime(2010, 7, 1, 0, 0): 7.8885369300842285,
 datetime.datetime(2010, 8, 1, 0, 0): 7.454631328582764,
 datetime.datetime(2010, 9, 1, 0, 0): 8.701157569885254,
 datetime.datetime(2010, 10, 1, 0, 0): 9.229513168334961,
 datetime.datetime(2010, 11, 1, 0, 0): 9.541375160217285,
 datetime.datetime(2010, 12, 1, 0, 0): 9.891261100769043,
 datetime.datetime(2011, 1, 1, 0, 0): 10.405207633972168,
 datetime.datetime(2011, 2, 1, 0, 0): 10.831138610839844,
 datetime.datetime(2011, 3, 1, 0, 0): 10.68701457977295,
 datetime.datetime(2011, 4, 1, 0, 0): 10.73669147491455,
 datetime.datetime(2011, 5, 1, 0, 0): 10.66616153717041,
 datetime.datetime(2011, 6

### <font color=blue>Note: stock splits</font>
<br>
Notice that stock splits happen, and it should be taken consideration when calculating the multi-year average of price multiples over certain income metrics, such as earnings, cash flows, etc.

In [21]:
aapl.splits

Date
2014-06-09    7.0
2020-08-31    4.0
Name: Stock Splits, dtype: float64